# Title

### **Can machine learning models accurately predict maternal health risk based on physiological features?**

---

## Summary

This project aims to use machine learning to predict maternal health risk based on key physiological factors. Maternal health are a major concern in healthcare. Hence, early risk assessment is crucial for the welfare of both the mother and child. By leveraging structured data from the Maternal Health Risk Dataset, we trained and evaluated machine learning models to classify maternal health risk levels into three different classes: low, mid, and high risk categories.

The dataset contains 1,014 records with features such as age, blood pressures, blood sugar levels, body temperature and heart rate. Our goal will be to determine whether machine learning algorithms could accurately predict risk levels based on these physiological markers and aid medical experts.

We will carry out exploratory data analysis, feature engineering and preprocessing to the dataset and then apply multiple classification models including logistic regression, decision trees,___________, We might also carry out hyperparameter optimization and finally evaluate their accuracy and other classification metrics such as precision, recall, F1-score, and AUC-ROC.

---

## Introduction


Maternal health is a critical area of healthcare, as complications during pregnancy and childbirth can have severe consequences for both mothers and newborns. Conditions such as hypertension, gestational diabetes, and abnormal heart rate patterns can indicate a higher risk of complications. Historically, risk assessment have been carried out by medical professionals that relied heavily on clinical expertise and constant monitoring. However, since the boom of machine learning, it began to offer a data-driven approach to enhance early detection and intervention, all while minimizing the involvement of overworked medical professionals.

The dataset that we will be using to develop this ML model will consist in analyzing the following features for each patient:
- `Age`: Age of the patient (in years).
- `SystolicBP`: Systolic Blood Pressure (mmHg).
- `DiastolicBP`: Diastolic Blood Pressure (mmHg).
- `BS (Blood Sugar Level)`: Blood sugar concentration (mmol/L).
- `BodyTemp`: Body temperature (°F).
- `HeartRate`: Heart rate (beats per minute).
- `RiskLevel`: The target variable, categorized into low risk, mid risk, and high risk.

---

## Methods & Results


- describe in written english the methods you used to perform your analysis from beginning to end that narrates the code the does the analysis.
- your report should include code which:
    - loads data from the original source on the web
    - wrangles and cleans the data from it’s original (downloaded) format to the format necessary for the planned classification or clustering analysis
    - performs a summary of the data set that is relevant for exploratory data analysis related to the planned classification analysis
    - creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned classification analysis
    - performs classification or regression analysis
    - creates a visualization of the result of the analysis
    
note: all tables and figure should have a figure/table number and a legend

---

## Discussion


- summarize what you found
- discuss whether this is what you expected to find?
- discuss what impact could such findings have?
- discuss what future questions could this lead to?

---

## References

- at least 4 citations relevant to the project (format is your choose, just be consistent across the references).